### Génération des embeddings

##### 1. Charger les données

In [42]:
import pandas as pd

data_clean = pd.read_csv('../data/processed/data_clean.csv')
data_with_mentions = pd.read_csv('../data/processed/data_with_mentions.csv')
data_balanced_undersampling = pd.read_csv('../data/processed/data_balanced_undersampling.csv')

##### 2. Séparer les données en ensembles d'entraînement et de test

In [43]:
from sklearn.model_selection import train_test_split

def divide_sets(data):
    X = data.drop('airline_sentiment', axis=1)
    y = data['airline_sentiment']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

    print('X_Train:', len(X_train))
    print('X_Test :', len(X_test))
    print('Y_Train:', len(y_train))
    print('Y_Test :', len(y_test))

    return [X_train, X_test, y_train, y_test]

- Data 1: data_clean

In [44]:
X_train_clean, X_test_clean, y_train_clean, y_test_clean = \
divide_sets(data_clean)

X_Train: 12410
X_Test : 2191
Y_Train: 12410
Y_Test : 2191


- Data 2: data_with_mentions

In [45]:
X_train_mentions, X_test_mentions, y_train_mentions, y_test_mentions = \
divide_sets(data_with_mentions)

X_Train: 12410
X_Test : 2191
Y_Train: 12410
Y_Test : 2191


- Data 3: data_balanced_undersampling

In [46]:
X_train_undersampling, X_test_undersampling, y_train_undersampling, y_test_undersampling = \
divide_sets(data_balanced_undersampling)

X_Train: 7254
X_Test : 1281
Y_Train: 7254
Y_Test : 1281


##### 3. Le choix de modèles de génération d’embeddings

Parmi les meilleurs modèles de génération d’embeddings, on trouve :

| Modèle              | Qualité         | Multilingue       | Vitesse |
| ------------------- | --------------- | ----------------- | ------- |
| BGE-large-m3        | Excellent       | Oui               | Lent |
| E5-large-v2         | Excellent       | EN only           | Moyen |
| Multilingual-E5     | Bien            | Oui               | Moyen |
| GTE-large           | Bien            | EN only           | Rapide |
| MiniLM-L12-v2       | qualité faible  | Oui               | Très rapide |


On va utiliser `E5-large-v2` pour trouver un équilibre entre la performance et la vitesse d’entraînement.

##### 4. Prétraitement des textes pour le modéle `E5-large-v2`

Le modèle `E5-large-v2` a été entraîné sur du texte naturel (phrases réelles, majuscules, ponctuation, etc.), donc il faut garder le texte aussi proche que possible de sa forme originale,
tout en nettoyant les erreurs mineures ou les incohérences.

il est recommander d'éviter de:
- Supprimer les stopwords: Le modèle comprend leur rôle sémantique.
- Lemmatisation ou stemming: Cela modifie la structure linguistique et le sens.
- Supprimer la ponctuation: La ponctuation aide à comprendre le contexte.
- Tout mettre en minuscules (Sauf si le texte contient des majuscules aléatoires ou tout en majuscules.): Peut faire perdre des distinctions importantes (Apple ≠ apple, US ≠ us).
- Supprimer les chiffres: Les nombres ont souvent une signification utile (dates, montants…).

Il est Recommander (et parfois obligatoire) de:
- Supprimer les espaces inutiles: Nettoyer les débuts/fin de lignes et les espaces multiples.
- Gérer les valeurs manquantes: Remplacer les `NaN` ou textes vides par une chaîne vide.
- Ajouter le préfixe (Obligatoire): **"passage: "** pour les documents, **"query: "** pour les requêtes.
- Tronquer les textes très longs: Si un texte dépasse la limite (512 tokens environ), on peut le couper.

##### 5. Charger le modèle `intfloat/e5-large-v2` avec Sentence Transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('intfloat/e5-large-v2')

##### 6. Générer les embeddings pour les données

In [48]:
import numpy as np

def generate_embeddings(X_train, X_test, suffix):

    # Générer les embeddings d'entrainement

    texts = X_train['text'].tolist()

    train_embeddings = model.encode(
        texts,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=True
    )

    # Générer les embeddings de test

    texts = X_test['text'].tolist()

    test_embeddings = model.encode(
        texts,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=True
    )

    # Sauvegarder les embeddings Générés (.npy)

    np.save(f'../data/embedding/train_embeddings_{suffix}.npy', train_embeddings)
    np.save(f'../data/embedding/test_embeddings_{suffix}.npy', test_embeddings)

    return [train_embeddings, test_embeddings]

- Data 1: data_clean

In [49]:
train_embeddings_clean, test_embeddings_clean = \
generate_embeddings(X_train_clean, X_test_clean, 'clean')

Batches: 100%|██████████| 69/69 [02:56<00:00,  2.56s/it]


- Data 2: data_with_mentions

In [50]:
train_embeddings_mentions, test_embeddings_mentions = \
generate_embeddings(X_train_mentions, X_test_mentions, 'mentions')

Batches: 100%|██████████| 69/69 [08:16<00:00,  7.19s/it]


- Data 3: data_balanced_undersampling

In [51]:
train_embeddings_undersampling, test_embeddings_undersampling = \
generate_embeddings(X_train_undersampling, X_test_undersampling, 'undersampling')

Batches: 100%|██████████| 41/41 [05:46<00:00,  8.44s/it]


##### 7. Afficher les embeddings Générés

In [52]:
print('Train - 1:', train_embeddings_clean.shape)
print('Test  - 1:', test_embeddings_clean.shape, '\n')
print('Train - 2:', train_embeddings_mentions.shape)
print('Test  - 2:', test_embeddings_mentions.shape, '\n')
print('Train - 3:', train_embeddings_undersampling.shape)
print('Test  - 3:', test_embeddings_undersampling.shape, '\n')

print(train_embeddings_clean)

Train - 1: (12410, 1024)
Test  - 1: (2191, 1024) 

Train - 2: (12410, 1024)
Test  - 2: (2191, 1024) 

Train - 3: (7254, 1024)
Test  - 3: (1281, 1024) 

[[ 0.02839299 -0.02863677  0.00565065 ... -0.02072558  0.0079131
   0.0486407 ]
 [ 0.03700669 -0.03339322  0.04501049 ... -0.03881466  0.0141934
   0.01910754]
 [ 0.00680762 -0.0303178   0.03598471 ... -0.03867556  0.01009593
   0.00223314]
 ...
 [ 0.02948683 -0.04877236  0.02602102 ... -0.04681333  0.02730181
   0.03831755]
 [ 0.01031082 -0.05356324  0.00806744 ... -0.02714521  0.03243184
   0.03135326]
 [ 0.0027516  -0.02729725  0.02865412 ... -0.03459664  0.03529878
   0.03521831]]


##### 8. Sauvegarder les labels et identifiants

In [53]:
from sklearn.preprocessing import LabelEncoder
import joblib as jb

def encode_labels(y_train, y_test, suffix):

    encoder = LabelEncoder()

    # Metadata de l'entrainement

    train_metadata = pd.DataFrame()
    train_metadata['label_name'] = y_train
    train_metadata['id'] = range(0, len(train_metadata))
    train_metadata['label'] = encoder.fit_transform(train_metadata['label_name'])

    # Metadata de test

    test_metadata = pd.DataFrame()
    test_metadata['label_name'] = y_test
    test_metadata['id'] = range(0, len(test_metadata))
    test_metadata['label'] = encoder.transform(test_metadata['label_name'])

    # Sauvegarder les metadatas

    train_metadata.to_csv(f'../data/metadata/train_metadata_{suffix}.csv', index=False)
    test_metadata.to_csv(f'../data/metadata/test_metadata_{suffix}.csv', index=False)

    # Sauvegarder l'encoder

    jb.dump(encoder, f'../models/encoders/encoder_{suffix}.pkl')

    # Afficher un message de réussi

    print("Les métadonnées ont été générées et enregistrées.")


- Data 1: data_clean

In [54]:
encode_labels(y_train_clean, y_test_clean, 'clean')

Les métadonnées ont été générées et enregistrées.


- Data 2: data_with_mentions

In [55]:
encode_labels(y_train_mentions, y_test_mentions, 'mentions')

Les métadonnées ont été générées et enregistrées.


- Data 3: data_balanced_undersampling

In [56]:
encode_labels(y_train_undersampling, y_test_undersampling, 'undersampling')

Les métadonnées ont été générées et enregistrées.
